## 19. 동적 속성과 프로퍼티
### 19.1 동석 속성을 이용한 데이터 랭글링
다음에 나올 몇 가지 예제에서는 OSCON 2014 콘퍼런스에서 오라일리가 공개한 JSON 데이터 피드를 사용하기 위해 동적 속성을 이용한다.

In [1]:
""" [예제 19-2] osconfeed.py: osconfeed.json 내려받기 """

from urllib.request import urlopen
import warnings
import os 
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg) # 새로 내려받아야 하는 경우 경고 메시지를 출력
        with urlopen(URL) as remote, open(JSON, 'wb') as local: # 두 개의 콘텍스트 관리자를 이용해서 원격 파일을 읽고 저장하는 with 문
            local.write(remote.read())
            
    with open(JSON) as fp:
        return json.load(fp) # JSON 파일을 파싱하고 네이티브 파이썬 객체로 반환하며, 이 피드에는 dict, list, str, int 형의 데이터가 있음

In [2]:
feed = load()
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [3]:
for key, value in sorted(feed['Schedule'].items()):
    print("{:3} {}".format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [4]:
feed['Schedule']['speakers'][-1]['name'] # 마지막 발표자의 이름

'Carina C. Zona'

### 19.1.1 동적 속성을 이용해서 JSON과 유사한 데이터 둘러보기
[예제 19-2]는 아주 간단하지만 feed['Schedule']['event'][40]['name']과 같은 구문은 번거롭다. 예제 19-5의 FrozenJSON을 사용하면 자바스크립트처럼 feed.Schedule.event[40].name과 같은 구문으로 동일한 값을 가져올 수 있다. 

FrozenJSON의 핵심은 \_\_getattr\_\_( ) 메서드다.  \_\_getattr\_\_( ) 특별 메서드는 속성을 가져오기 위한 일반적인 과정이 실패할 때(즉, 지명한 속성을 객체, 클래스, 슈퍼클래스에서 찾을 수 없을 때)만 인터프리터에서 호출한다.

In [5]:
""" [예제 19-5] explore0.py: JSON 데이터셋을 내포한 FrozenJSON 객체, 리스트, 기본형을 담고 있는 FrozenJSON으로 변환 """

from collections import abc

class FrozenJSON:
    """
    점 표기법을 이용해서 JSON과 유사한 객체를 순회하는 읽기 전용 퍼사드 클래스
    """
    
    def __init__(self, mapping):
        self.__data = dict(mapping) # 딕셔너리 메서드를 사용할 수 있으며, 원본을 변경하지 않는다.
        
    def __getattr__(self, name): # name 속성이 없을 때만 __getattr__() 메서드가 호출된다.
        if hasattr(self.__data, name):
            return getattr(self.__data, name) # __data에 들어 있는 객체가 name 속성을 가지고 있으면 그 속성을 반환한다.
        else:
            return FrozenJSON.build(self.__data[name])  # 그렇지 않으면 self.__data에 name을 키로 사용해서 항목을 가져오고 가져온 항목에
                                                        # FrozenJSON.build()를 호출한 결과를 반환한다.
        
    @classmethod # 일반적으로 대안 생성자로 @classmethod를 사용한다.
    def build(cls, obj):
        if isinstance(obj, abc.Mapping): # obj가 매핑형이면 이 객체로부터 FrozenJSON 객체를 생성한다.
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): # obj가 MutableSequence 형이면 리스트이므로 obj안에 있는 모든 항목에 build() 메서드를 적용해서 생성된 객체들의 리스트를 반환한다.
            return [cls.build(item) for item in obj]
        else:
            return obj # obj가 매핑도 아니고 리스트도 아니면 항목을 그대로 반환한다.

In [6]:
raw_feed = load()
feed = FrozenJSON(raw_feed)
len(feed.Schedule.speakers)

357

In [7]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [8]:
for key, value in sorted(feed.Schedule.items()):
    print("{:3} {}".format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [9]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [10]:
talk = feed.Schedule.events[40]
type(talk) # JSON 객체였지만 이제는 FrozenJSON 객체가 되었다.

__main__.FrozenJSON

In [11]:
talk.name

'There *Will* Be Bugs'

In [12]:
talk.speakers

[3471, 5199]

In [13]:
talk.flavor # 없는 속성을 읽으려 시도하면 일반적으로 발생하는 AttributeError 대신 KeyError 예외가 발생한다.

KeyError: 'flavor'

### 19.1.2 잘못된 속성명 문제
FrozenJSON 클래스는 파이썬 키워드가 속성명으로 사용된 경우를 처리하지 못한다. 때문에 전달된 매핑 안의 키가 파이썬 키워드인지 검사하고, 파이썬 키워드인 경우에는 뒤에 \_를 붙여 속성을 읽을 수 있게 만드는 것이 좋다.

In [14]:
""" [예제 19-6] 파이썬 키워드인 속성명에 언더바 붙이기 """

def __init__(self, mapping):
    self.__data = {}
    for key, value in mapping.items():
        if keyword.iskeyword(key):
            key += '_'
        self.__data[key] = value

In [15]:
x = FrozenJSON({'2be':'or not'})

In [16]:
x.2be

SyntaxError: invalid syntax (<ipython-input-16-8694215ab5bd>, line 1)

### 19.1.3 \_\_new\_\_( )를 이용한 융통성 있는 객체 생성
이전에 build() 클래스에서 처리하던 논리를 \_\_new\_\_( ) 메서드로 옮긴 새로운 버전의 FronzenJson 클래스를 보여준다.

In [17]:
""" [예제 19-7] FronzenJson 객체든 아니든 새로운 객체를 생성하는 대신 __new__() 사용하기 """

from collections import abc
from keyword import iskeyword

class FrozenJSON:
    """
    점 표기법을 이용해서 JSON과 유사한 객체를 둘러보기 위한 읽기 전용 퍼사드 클래스
    """
    
    def __new__(cls, arg): 
        """
        클래스 메서드로서 첫번째 인수는 클래스 자신, 나머지 인수는 __init__() 과 동일하다.
        """
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls) # 기본적으로 슈퍼클래스에 위임한다. 
                                        # 이 경우 FrozenJSON을 인수로 전달하고 object 클래스에서 __new__() 메서드를 호출한다. 
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg
        
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if iskeyword(key):
                key += '_'
            self.__data[key] = value
            
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name]) # 생성자 호출

In [18]:
raw_feed = load()
feed = FrozenJSON(raw_feed)
len(feed.Schedule.speakers)

357

In [19]:
for key, value in sorted(feed.Schedule.items()):
    print("{:3} {}".format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


#### 19.1.4 shelve를 이용해서 OSCON 피드 구조 변경하기
shelve.open() 고위 함수는 shelve.Shelf 객체를 반환한다. shelve.Shelf는 dbm 모듈을 이용해서 키-값 객체를 보관하는 단순한 객체로서 아래과 같은 특징이 있다. 우리는 Json 파일에서 레코드를 모두 읽어 shelve.Shelf에 저장할 것이다. 각 키는 'event.33950' 처럼 레코드 유형과 일련번호로 만들며, 값은 이제 설명할 대로 만든 Record 클래스 객체가 된다.

+ shelve.Shelf는 abc.MutableMapping 클래스를 상속하므로, 매핑형이 제공하는 핵심 메서드들을 제공한다.
+ shelve.Shelf는 sync(), close() 등의 입출력을 관리하는 메서드도 제공한다. 콘텍스트 관리자이기도 하다.
+ 새로운 값이 키에 할당될 때마다 기와 값이 저장된다.
+ 깂은 반드시 pickle 모듈이 처리할 수 있는 객체여야 한다.

In [35]:
""" [예제 19-9] shedule1.py: shelve.Shelf에 저장된 OSCON 일정 데이터 둘러보기 """
import warnings

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs) # 키워드 인수로부터 생성된 속성으로 객체를 생성할 때 간편히 사용하는 방법
        
def load_db(db):
    raw_data = load()
    warnings.warn("loading" + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1] # 키값의 마지막 s자를 제거한다.(confrerence, event, speaker, venue)
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial']) 
            record['serial'] = key
            # print(record) # for debugging
            db[key] = Record(**record) # Recode 객체를 생성하고, 해당 key로 데이터베이스에 저장한다.

In [36]:
import shelve

db = shelve.open(DB_NAME)
if CONFERENCE not in db:
    load_db(db)

In [37]:
speaker = db['speaker.3471']
type(speaker)

__main__.Record

In [38]:
speaker.name, speaker.twitter

('Anna Ravenscroft', 'annaraven')

In [40]:
db.close()

Record.\_\_init\_\_() 메서드는 널리 사용되는 파이썬 꼼수를 보여준다. \_\_slots\_\_ 속성이 클래스에 선언되어 있지 않은 한 객체의 \_\_dict\_\_에 속성들이 들어 있다. 따라서 객체의 \_\_dict\_\_를 직접 매핑형으로 설정하면, 그 객체의 속성 묶음을 빠르게 정의할 수 있다.

### 19.1.5 프로퍼티를 이용해서 연결된 레코드 읽기
shelf에서 가져온 event 레코드의 venue나 speakers 속성을 읽을 때 일련번호 대신 온전한 레코드 객체를 반환하는 것이 이번 버전의 목표이다. 

In [41]:
""" [예제 19-11] schedule2.py: 임포트, 상수, 개선된 Record 클래스 """
import warnings
import inspect

DB_NAME = 'data/schedule2_db'
CONFERENCE = 'conference.115'

class Record:
    """
    기존 Record 클래스에 
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    
    def __eq__(self, other):
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

In [43]:
""" [예제 19-12] schedule2.py: MissingDatabaseError와 DbRecord 클래스 """
class MissingDatabaseError(RuntimeError):
    """
    필요한 데이터베이스가 설정되어 있지 않을 때 발생
    """
    
class DbRecord(Record):
    
    __db = None
    
    @staticmethod
    def set_db(db):
        DbRecord.__db = db
        
    @staticmethod
    def get_db():
        return DbRecord.__db
    
    @classmethod
    def fetch(cls, ident): # 클래스 메서드이므로 서브 클래스에서 쉽게 커스터마이즈할 수 있다.
        db = cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not set; call '{}.setdb(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:
                raise # 나머지 예외는 처리할 수 없으므로 다시 발생 
    
    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name = self.__class__.__name__
            return "<{} serial={!r}>".format(cls_name, self.serial)
        else:
            return super().__repr__() # serial 속성이 없으면 슈퍼클래스의 __repr__() 메서드를 사용한다. 

In [ ]:
"""  """

In [ ]:
DbRecord.set_db(db) # DbRecord는 Record를 상속해서 데이터베이스를 지원한다. 
event = DbRecord.fetch('event.33950') # 어떠한 종류의 레코드든 관계없이 가져온다.
event # DbRecord 클래스를 상속한 Event클래스 객체다.

In [ ]:
event.venue # DbRecord 객체가 반환된다.

In [ ]:
event.venue.name # 자동으로 dereference 하는 것이 이 예제의 목표이다. 

In [ ]:
for spkr in event.speakers: # event.speakers 리스트를 반복해서, 각 발표자를 나타내는 DbRecord 객체들도 가져올 수 있다.
    print('{0.serial}: {0.name}'.format(spkr))